In [1]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 

# ZALORA Scraping

In [2]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
# Scraping the page
def scrape_detail_page(driver,category,collection):
    # scraping page
    path = driver.find_elements_by_class_name('b-catalogList__itm')
    result = pd.DataFrame()
    len_path = round(len(path)/2)
    for x in range(len_path):
        element = path[x*2]
        element.location_once_scrolled_into_view
        
        brand = driver.find_elements_by_class_name('js-catalogProductTitle')[x].text
        name = driver.find_elements_by_class_name('b-catalogList__itmTitle')[x].text
        try:
            origin_price = re.sub("\.","",re.sub("Rp ","",path[x*2].find_elements_by_class_name('fixed-price')[0].text))
        except:
            try:
                origin_price = re.sub("\.","",re.sub("Rp ","",path[x*2].find_elements_by_css_selector('.b-catalogList__itmPrice.old.old-price')[0].text))
            except:
                origin_price = 0
        try:
            discounted_price = re.sub("\.","",re.sub("Rp ","",path[x*2].find_elements_by_class_name('special-price')[0].text))
        except:
            discounted_price = origin_price
        if(origin_price==0):
            origin_price = discounted_price

        image = re.sub(" 2x","",driver.find_elements_by_class_name('js-catalogImage')[x].find_element_by_xpath('img').get_attribute('srcset'))
        while(len(image)<1):
            image = re.sub(" 2x","",driver.find_elements_by_class_name('js-catalogImage')[x].find_element_by_xpath('img').get_attribute('srcset'))
            
        sku = driver.find_elements_by_class_name('js-catalogImage')[x].get_attribute('data-sku')

        # Return result
        date = datetime.datetime.now().strftime("%Y-%m-%d")
        new_result = pd.DataFrame([{'brand':brand,'sku':sku,'name':name, 'origin_price':origin_price, 'discounted_price':discounted_price, 'image':image,'category':category,'scraping_date':date}])
        result = result.append(new_result)
        #print(new_result)
        
    # save to mongodb
    try:
        collection.insert_many(result.to_dict('records'))
    except:
        PrintException()
        pass

    #driver.close()
    return result

## By brand

In [4]:
def scrape_url(url,category,collection):
        
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.zalora_brand
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)

    # Start Scraping
    result = pd.DataFrame()
    scrape = True
    page = 1
    while(scrape):
        # Go to catalogue page
        result = result.append(scrape_detail_page(driver,category,collection))
        # Go to the next page
        try:
            check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]
            page = page+1
            next_page = url + "&page=" + str(page)
            print("Go to next page: " + next_page)
            driver.close()
            driver = webdriver.Firefox()
            driver.get(next_page)
        except:
            PrintException()
            scrape = False
    
    driver.close()
    return result


In [ ]:
# MAIN
zalora_category = pd.read_csv('C:\\Users\\Annisa\\Stream\\Product Development - Analysis - QBR Jul19\\Data\\scraping\\zalora_category.csv',sep=",")
zalora_category = zalora_category[zalora_category.collection.isin(['sepatu_wanita','sepatu_pria'])]
result = pd.DataFrame()
for x in range(0,len(zalora_category)):
    category = zalora_category.iloc[x]
    print(category['category'])
    
    # change brand
    url = category['url']+"?brand=bata"
    
    result = scrape_url(url,category['category'],category['collection'])
    print(result.head())

In [5]:
# MAIN
zalora_category = pd.read_csv('C:\\Users\\tamu\\SharePoint\\Product Development - Documents\\Merchandising Analytics\\Analysis - QBR Jul19\\Data\\scraping\\zalora_category.csv')
result = pd.DataFrame()
for x in range(0,len(zalora_category)):
    category = zalora_category.iloc[x]
    print(category['category'])
    
    # change brand
    url = category['url']+"?brand=mango"
    
    result = scrape_url(url,category['category'],category['collection'])
    print(result.head())

dress > party_dress
Connected successfully!!!
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand             category discounted_price  \
0  MANGO  dress > party_dress           650900   
0  MANGO  dress > party_dress           650900   
0  MANGO  dress > party_dress           479000   
0  MANGO  dress > party_dress           683900   
0  MANGO  dress > party_dress           630900   

                                               image  \
0  https://dynamic.zacdn.com/fkQyP5sDhgA8KxB-Sj3R...   
0  https://dynamic.zacdn.com/ri1g6GRcdhmlMI_ukFq_...   
0  https://dynamic.zacdn.com/l28Ra4JyVh2Q6ez85PZe...   
0  https://dynamic.zacdn.com/ACW1juaIvIXn4V8Yoj_-...   
0  https://dynamic.zacdn.com/lsOO_yAREx90RZDt0LD-...   

                          name origin_price scraping_date               sku  
0  Embroidered Checkered Dr

EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand            category discounted_price  \
0  MANGO  dress > long_dress           783900   
0  MANGO  dress > long_dress           632900   
0  MANGO  dress > long_dress           889900   
0  MANGO  dress > long_dress           884900   
0  MANGO  dress > long_dress          1199000   

                                               image  \
0  https://dynamic.zacdn.com/qHx2EdhHHSI4iVk5Hmsr...   
0  https://dynamic.zacdn.com/Hf-IUARR9c_qOqCMYZx7...   
0  https://dynamic.zacdn.com/UO8U0V2pMEap6Dcxcp6e...   
0  https://dynamic.zacdn.com/oyI_gXQL4yqkR1r_TGwg...   
0  https://dynamic.zacdn.com/Nh0qH3lWYMig50RzM0XT...   

                       name origin_price scraping_date               sku  
0        Floral Print Dress      1199000    2019-05-27  57C88AA2F75816GS  
0      

Go to next page: https://www.zalora.co.id/women/pakaian/rok-midi/?brand=mango&page=2
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand        category discounted_price  \
0  MANGO  rok > rok_midi           427900   
0  MANGO  rok > rok_midi           676900   
0  MANGO  rok > rok_midi           543900   
0  MANGO  rok > rok_midi           771900   
0  MANGO  rok > rok_midi           699000   

                                               image  \
0  https://dynamic.zacdn.com/sOtDMGi3jFUxAX9frdD3...   
0  https://dynamic.zacdn.com/TW3D0tYH53bGLSrkfXB4...   
0  https://dynamic.zacdn.com/Qh-sjkx7U87_6WewM9Ez...   
0  https://dynamic.zacdn.com/dAn4YkIDIuEkbXLGHlZl...   
0  https://dynamic.zacdn.com/gDp0w5RT7hXwut2F1H1O...   

                           name origin_price scraping_date               sku  
0            Chec

Go to next page: https://www.zalora.co.id/women/pakaian/straightleg-pants/?brand=mango&page=2
Go to next page: https://www.zalora.co.id/women/pakaian/straightleg-pants/?brand=mango&page=3
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand                              category discounted_price  \
0  MANGO  celana_legging > celana_straight_leg           369000   
0  MANGO  celana_legging > celana_straight_leg           775900   
0  MANGO  celana_legging > celana_straight_leg           356900   
0  MANGO  celana_legging > celana_straight_leg           585900   
0  MANGO  celana_legging > celana_straight_leg           609900   

                                               image                     name  \
0  https://dynamic.zacdn.com/hCOPGawerqejolIKAoVl...   Crop Slim-Fit Trousers   
0  https://dynamic.zacdn.com/npA10kK

EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
jeans > flare_jeans
Connected successfully!!!
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand             category discounted_price  \
0  MANGO  jeans > flare_jeans           699000   
0  MANGO  jeans > flare_jeans           524900   
0  MANGO  jeans > flare_jeans           481900   
0  MANGO  jeans > flare_jeans           699000   
0  MANGO  jeans > flare_jeans           658900   

                                               ima

...

Empty DataFrame
Columns: []
Index: []
flats > peep_toes
Connected successfully!!!
EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
flats > sepatu_loafers_and_boat
Connected successfully!!!
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand                         category discounted_price  \
0  MANGO  flats > sepatu_loafers_and_boat           478900   
0  MANGO  flats > sepatu_loafers_and_boat           628900   
0  MANGO  flats > sepatu_loafers_and_boat           999000   

        

   brand               category discounted_price  \
0  MANGO  sandals > espadrilles           473900   

                                               image                name  \
0  https://dynamic.zacdn.com/Ic2njtvf1nye-YBIpHzs...  Sueded Espadrilles   

  origin_price scraping_date               sku  
0       699000    2019-05-27  2B921SH0AC7FE0GS  
sandals > sandal_flats
Connected successfully!!!
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand                category discounted_price  \
0  MANGO  sandals > sandal_flats           398900   
0  MANGO  sandals > sandal_flats           640900   
0  MANGO  sandals > sandal_flats           442900   
0  MANGO  sandals > sandal_flats           444900   
0  MANGO  sandals > sandal_flats           475900   

                                               image  \
0  https:

EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
sneakers
Connected successfully!!!
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
   brand  category discounted_price  \
0  MANGO  sneakers           418900   
0  MANGO  sneakers           658900   
0  MANGO  sneakers           999000   
0  MANGO  sneakers           578900   
0  MANGO  sneakers           848900   

                                               image  \
0  https://dynamic.zacdn.com/eh3zro4Pg6hGqh39fVRj...   
0  https://dyna

Empty DataFrame
Columns: []
Index: []
kemeja > kemeja_formal
Connected successfully!!!
EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
kemeja > kemeja_batik
Connected successfully!!!
EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
polo_shirt
Connected successfully!!!
EXCEPTION IN (<ipython-input-3-e85132a7c79b>

Empty DataFrame
Columns: []
Index: []
beachwear
Connected successfully!!!
EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
underwear_and_loungewear > pakaian_tidur
Connected successfully!!!
EXCEPTION IN (<ipython-input-3-e85132a7c79b>, LINE 41 "collection.insert_many(result.to_dict('records'))"): documents must be a non-empty list
EXCEPTION IN (<ipython-input-4-3c5a02666d29>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
underwear_and_loungewear > pakaian_dalam
Connected successfully!!!
EXCEPTIO

## Top 500 per category

In [4]:
def scrape_url2(url,category,collection):
        
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.zalora
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)

    # Start Scraping
    result = pd.DataFrame()
    scrape = True
    page = 1
    start = 0
    while(scrape and page<8):
        # Go to catalogue page
        result = result.append(scrape_detail_page(driver,category,collection))
        # Go to the next page
        try:
            check_next = driver.find_elements_by_xpath('/html/body/div[2]/div[5]/div[4]/div[3]/div[4]/div[2]/div/div/div/div/ul/li[3]/a')[0]
            page = page+1
            next_page = url + "&page=" + str(page)
            print("Go to next page: " + next_page)
            driver.close()
            driver = webdriver.Firefox()
            driver.get(next_page)
        except:
            PrintException()
            scrape = False
    
    driver.close()
    return result


In [5]:
# MAIN
zalora_category = pd.read_csv('C:\\Users\\Annisa\\Stream\\Product Development - Analysis - QBR Jul19\\Data\\scraping\\zalora_category.csv',sep=",")
result = pd.DataFrame()
for x in range(100,len(zalora_category)):
    category = zalora_category.iloc[x]
    print(category['category'])
    
    # change brand
    url = category['url']+"?sort=popularity"
    collection = category['collection']+"_top"
    
    result = scrape_url2(url,category['category'],collection)
    print(result.head())

sneakers
Connected successfully!!!
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=2
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=3
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=4
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=5
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=6
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=7
Go to next page: https://www.zalora.co.id/men/sepatu/sneakers/?sort=popularity&page=8
           brand  category discounted_price  \
0        Charlie  sneakers           189500   
0      Dr. Kevin  sneakers           279960   
0   Hush Puppies  sneakers           399500   
0       Converse  sneakers           559000   
0  Blax Footwear  sneakers           174500   

                                               image  \
0  https://dynamic.zacdn

Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=2
Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=3
Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=4
Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=5
Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=6
Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=7
Go to next page: https://www.zalora.co.id/men/sepatu/sandals/?sort=popularity&page=8
        brand                       category discounted_price  \
0    Cardinal  sandal_and_flip_flop > sandal           284925   
0    Lvnatica  sandal_and_flip_flop > sandal           235000   
0    Lvnatica  sandal_and_flip_flop > sandal           235000   
0      CARVIL  sandal_and_flip_flop > sandal           176120   
0  Quiksilver  sandal_and_flip_flop > sandal           399000   

              